In [3]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                """Alexandru most  dangerous ensemble """
import os
__author__ = 'Daia Alexandru'
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.metrics import r2_score
from __future__ import division
import pandas as pd 
import  random as random
import numpy as np
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
import heapq
from sklearn.linear_model import  LinearRegression,BayesianRidge
from sklearn.ensemble import  RandomForestRegressor,RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.ensemble import *
import math
from sklearn import metrics
from sklearn.tree import *
 
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
 
 
class BadErrorTreshold(Exception):
    pass
 
class EnsembleRegressors(object):
    ''''
    ensembleRegressor is a class of ensembles generated by a  fix number of  iterations.This class is abstract 
    meaning  it  is not possible  to create instances  of it.,,stochasticLearning''  has interface  role. '''
    def __init__(self,regressors,sampling,iterations,test,X,y,k_fold):
        ''''intialize ensemble with  near all  parameters- see that   some of them are  intialized  by other methods'''
        self.sampling=sampling#dropout(cv) ratio-percentage
        if (self.sampling<=0.0 or self.sampling>=1.0):
            raise ValueError('Sampling  must be percent like value between[0,1) ex 0.75, you introduced :',self.sampling)
 
 
        self.test=test#final test set
        self.X=X#original train features
        self.y=y#original train outcome
        self.iterations=iterations#number of  iterations in the  stochastic ensemble
        self.regressors=regressors#ensemble models  for  each iteration      
        self.randomIndexes=[ [] for rounds in range(self.iterations)]#filled in with random dropout  by getStochasticDataSets()
        self.error=[]#error for  each averaged iteration
 
 
        self.stochasticPredictions=[ [] for  rounds in range(self.iterations)]#predictions on random drop outs
        self.finalPredictions=[]
 
        self.x_train=[]#his   next  4 rows are intialized  by getStochasticDataSets() according to random chunks
        self.y_train=[]
        self.x_test=[]
        self.y_test=[]
 
        self.new_x_ts=[]
        self.new_x_tr=[]
        self.new_test_final=[]
        self.k_folds=k_fold
        
    def returnError(self):
        '''returns : error for  each iteration-just like cross-validation'''
        return self.error
    
    def returnRandomIndexes(self):
        '''returns: random  indexes  for  each dropout'''
        return self.randomIndexes
    
    def returnstochasticPredictions(self):
        '''returns:predictions  on random dropous'''
        return self.stochasticPredictions
    
    def getStochasticDataSets(self):
        '''
        For some  potentially  iteration creates a random  idx variable representing the index  for random  dropout.
        Splits the train in  train/test(x_train,y_train,features and outcome for cross val train-x_test,y_test for cross_val test
        .Updated this train-test  variables with corresponding  values.See that  the initial vales are empty in the __init__.
        returns: idx(index of  dropout rows).
        
        '''
        x_train, x_test, y_train, y_test = train_test_split(self.X, self.y, test_size=self.sampling)
        self.x_train=x_train
        self.x_test=x_test
        self.y_train=y_train
        self.y_test=y_test
        #train_i, test_i = train_test_split(np.arange(self.X.shape[0]), train_size = self.sampling)
        #self.x_train=self.X[train_i]
        #self.y_train=self.y[train_i]
        #self.x_test=self.X[test_i]
        #self.y_test=self.y[test_i]
        #idx=test_i
        #return  idx
    
    def training(self,i):
        ''''
        For a  particular iteration ,,i''  calls getStochasticDataSets() method.In this was the randomIndexes feature is
        fiiled with corresponding index dropouts at position ,,i''.Since train/test new data sets according to cross validation
        are   filled  by getStochasticDataSets()  independet  of iteration number , this data sets will update for each new iter.
        performmed in stochasticLearning().'''
        self.getStochasticDataSets()
        #blending- we have x_train,y_train,x_test,y_test   our stochastic  dropouts for diversity
        #we are  going first  to make blend train,blend test from this dropoutsthen will see evaluation on the dropout
        clfs=self.regressors
        n_folds=self.k_folds
        skf=list(StratifiedKFold(self.y_train,n_folds))
        blend_tr=np.zeros((self.x_train.shape[0],len(clfs)))
        blend_ts=np.zeros((self.x_test.shape[0],len(clfs)))#for loca al evaluation
        blend_final_test=np.zeros((self.test.shape[0],len(clfs)))
        from sklearn.metrics  import  log_loss
        for  j , clf in  enumerate(clfs):
            print(clf)
            blend_ts_j=np.zeros((self.x_test.shape[0],len(skf)))
            blend_final_j=np.zeros((self.test.shape[0],len(skf)))
            for i ,(tr,ts) in enumerate(skf):
                print("fold",i)
                x_tr=self.x_train[tr]
                y_tr=self.y_train[tr]
                x_ts=self.x_train[ts]
                y_ts=self.y_train[ts]
                clf.fit(x_tr,y_tr)
                y_sub=clf.predict_proba(x_ts)[:,1]
                blend_tr[ts,j]=y_sub
                blend_ts_j[:,i]=clf.predict_proba(self.x_test)[:,1]
                blend_final_j[:,i]=clf.predict_proba(self.test)[:,1]
            blend_ts[:,j]=blend_ts_j.mean(1)
            blend_final_test[:,j]=blend_final_j.mean(1)
        #blend on evaluation
        import  xgboost as  xgb
        clf = xgb.XGBClassifier(n_estimators=1)
        clf.fit(blend_tr, self.y_train)
        
        pred_eval=  clf.predict_proba(blend_ts)[:,1]
        self.error.append(log_loss(self.y_test,pred_eval))
        #blend  on final test set:
        final_sub=clf.predict_proba(blend_final_test)[:,1]
        self.finalPredictions.append(final_sub)
        
        
 
        return self
    
    def getFinalPrediction(self,errorTreshold):
       raise NotImplementedError###averages models  according to best top prediction  trehsold  with different implementantions
    
    
        
    
    def stochasticLearning(self):
         raise NotImplementedError#Since EnsemebleRegressors class is abstract this will act  as an interface
                                  #and will be implemented   by  this  EnsembleRegressors sublclasses  for averaging and
                                  #weighted average
    
        
class AveragingModels(EnsembleRegressors):
    '''Inherits everything from EnsembleRegressors  class and implements stochastiLearning method  averaging models 
    for each iteration'''
 
    def stochasticLearning(self):
        
        
        for iter in range(self.iterations):
            print('Stochastic Iteration number ',iter)
            self.training(iter)
 
    def getFinalPrediction(self,errorTreshold):
        if (errorTreshold<=self.iterations and errorTreshold>0):
            
            topErrorIndex=heapq.nsmallest(errorTreshold, range(len(self.error)), self.error.__getitem__)
            finalAvg=0
            for topError in topErrorIndex:
                finalAvg=finalAvg+self.finalPredictions[topError]
            finalAvg=(finalAvg)/float(len(topErrorIndex))
            return finalAvg
        else:
            raise BadErrorTreshold
       
       
    
                                                


In [16]:
import  pandas  as  pd
import numpy  as np
train=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/numerai_training_data.csv')
print(train.head(3))
train1=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/trainCat.csv')
train['sums']=train1['sums']
train['kinetic']=train1['kinetic']
y=train['target']
train=train.drop(['target'],axis=1)
print(train.shape)
print(y.shape)
test=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/numerai_tournament_data.csv')
test1=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/testCat.csv')
test['sums']=test1['sums']
test['kinetic']=test1['kinetic']

id=test['t_id']
test=test.drop(['t_id'],axis=1)
sub=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/example_predictions.csv')
trainvec=np.array(train)
testvec=np.array(test)
y=np.array(y)

   feature1  feature2  feature3  feature4  feature5  feature6  feature7  \
0  0.185868  0.895807  0.596433  0.028381  0.053369  0.202876  0.912054   
1  0.709165  0.967550  0.529121  0.611704  0.961349  0.786213  0.489511   
2  0.526553  0.015789  0.940873  0.867239  0.084545  0.033899  0.412348   

   feature8  feature9  feature10   ...    feature13  feature14  feature15  \
0  0.669988  0.143481   0.132973   ...     0.855909   0.842887   0.913296   
1  0.607460  0.715500   0.476030   ...     0.517913   0.279985   0.938501   
2  0.538480  0.122290   0.370455   ...     0.992078   0.878373   0.090039   

   feature16  feature17  feature18  feature19  feature20  feature21  target  
0   0.938493   0.393211   0.211985   0.337367   0.898316   0.245452       0  
1   0.490024   0.630741   0.117521   0.917893   0.894724   0.897643       1  
2   0.379432   0.271485   0.361007   0.379741   0.028594   0.351007       0  

[3 rows x 22 columns]
(136573, 23)
(136573,)


In [50]:
from sklearn.metrics import log_loss
log_loss(y,avg)

0.71520595524594721

In [62]:
avg=np.average(trainvec[:,1:13],axis=1)
avg2=np.average(trainvec[:,6:21],axis=1)
avg3=(avg+avg2)/1
from sklearn.metrics import log_loss
log_loss(y,avg3)

9.1564793204614645

In [5]:
#sub=pd.read_csv('/home/machine_learning/Downloads/numerai_datasets/example_predictions.csv')


In [5]:
import xgboost  as xgb
from sklearn.ensemble import RandomForestClassifier

In [26]:
""" run stuff with  0.1 dropout , 3 iterations,predict on testvec, train on trainvec and  y ,do 10
stratified  sampling"""
import  xgboost  as xgb
clfs=[ xgb.XGBClassifier(n_estimators=10),xgb.XGBClassifier(n_estimators=5)
      ,xgb.XGBClassifier(n_estimators=15),RandomForestClassifier(n_estimators=5),
      
 
      RandomForestClassifier(n_estimators=15)
 
 
 
     ]
o=AveragingModels(clfs,0.8,3,trainvec,trainvec,y,2)
o.stochasticLearning()


Stochastic Iteration number  0
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=10, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
fold 0
fold 1
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=5, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
fold 0
fold 1
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=15, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       sca

In [27]:
o.error

[0.69298162027794008, 0.69294892777155759, 0.69298553920791373]

In [28]:
pred=o.getFinalPrediction(3)
pred

array([ 0.50028318,  0.5030815 ,  0.49539718, ...,  0.50267196,
        0.49669173,  0.50028318], dtype=float32)

In [72]:
#sub['probability']=pred
#sub.head(3)

,t_id,probability
0,3234,0.492083
1,2161,0.492083
2,3927,0.498085


In [73]:
#sub.to_csv('/home/machine_learning/Downloads/sub10.csv',index=False)

In [23]:
from gplearn.genetic import SymbolicTransformer
function_set = ['add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg', 'inv', 'max', 'min']
gp = SymbolicTransformer(generations=10, population_size=200,
                         hall_of_fame=30, n_components=20,
                       
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0)

In [ ]:
gp.fit(trainvec, pred)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    10.36   0.213601638207        3    0.59285929376   0.589549415289      2.71m
   1     8.56   0.430897286581        6   0.607727984482    0.61046670946      2.41m
   2     4.92   0.515956360952       16   0.628716658074   0.636893727495      2.09m
   3     7.96   0.498925426986        8   0.641193633833   0.625300060281      1.81m
   4     9.82   0.506271324568       12   0.692093847797   0.690452083308      1.52m
   5    10.92   0.523466323809       12   0.692112504267     0.6901428501      1.23m


In [14]:
new_tr=gp.transform(trainvec)

In [16]:
new_ts=gp.transform(testvec)

In [17]:
other_train=np.hstack((trainvec,new_tr))
other_test=np.hstack((testvec,new_ts))

In [18]:
other_train.shape

(136573, 26)

In [20]:
""" run stuff with  0.1 dropout , 3 iterations,predict on testvec, train on trainvec and  y ,do 10
stratified  sampling"""
import  xgboost  as xgb
clfs=[ xgb.XGBClassifier(n_estimators=10),xgb.XGBClassifier(n_estimators=5)
      ,xgb.XGBClassifier(n_estimators=15),RandomForestClassifier(n_estimators=5),
      
 
      RandomForestClassifier(n_estimators=15)
 
 
 
     ]
o=AveragingModels(clfs,0.8,3,other_test,other_train,y,3)
o.stochasticLearning()


Stochastic Iteration number  0
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=10, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
fold 0
fold 1
fold 2
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=5, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
fold 0
fold 1
fold 2
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=15, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda

In [21]:
o.error

[0.69295196932578629, 0.69294682301605526, 0.69292343683235114]

In [22]:
pred=o.getFinalPrediction(1)
pred
sub['probability']=pred
sub.to_csv('/home/machine_learning/Downloads/sub11.csv',index=False)